In [25]:
import numpy as np, pandas as pd, os
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn import svm, neighbors
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from tqdm import tqdm_notebook

In [26]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.head()

,id,muggy-smalt-axolotl-pembus,dorky-peach-sheepdog-ordinal,slimy-seashell-cassowary-goose,snazzy-harlequin-chicken-distraction,frumpy-smalt-mau-ordinal,stealthy-beige-pinscher-golden,chummy-cream-tarantula-entropy,hazy-emerald-cuttlefish-unsorted,nerdy-indigo-wolfhound-sorted,leaky-amaranth-lizard-sorted,ugly-tangerine-chihuahua-important,shaggy-silver-indri-fimbus,flaky-chocolate-beetle-grandmaster,squirrely-harlequin-sheep-sumble,freaky-tan-angelfish-noise,lousy-plum-penguin-sumble,bluesy-rose-wallaby-discard,baggy-copper-oriole-dummy,stealthy-scarlet-hound-fepid,greasy-cinnamon-bonobo-contributor,cranky-cardinal-dogfish-ordinal,snippy-auburn-vole-learn,greasy-sepia-coral-dataset,flabby-tangerine-fowl-entropy,lousy-smalt-pinscher-dummy,bluesy-brass-chihuahua-distraction,goopy-eggplant-indri-entropy,homey-sepia-bombay-sorted,homely-ruby-bulldog-entropy,hasty-blue-sheep-contributor,blurry-wisteria-oyster-master,snoopy-auburn-dogfish-expert,stinky-maroon-blue-kernel,bumpy-amaranth-armadillo-important,slaphappy-peach-oyster-master,dorky-tomato-ragdoll-dataset,messy-mauve-wolverine-ordinal,geeky-pumpkin-moorhen-important,crabby-teal-otter-unsorted,flaky-goldenrod-bat-noise,beady-aquamarine-affenpinscher-grandmaster,pasty-auburn-rat-entropy,snappy-gamboge-caterpillar-dummy,breezy-persimmon-newt-noise,baggy-mustard-collie-hint,sunny-red-weasel-hint,cheeky-red-armadillo-unsorted,wiggy-magnolia-wolfhound-sorted,flimsy-chestnut-guppy-important,beady-champagne-bullfrog-grandmaster,lanky-emerald-alligator-kernel,blurry-flax-sloth-fepid,scanty-viridian-guppy-grandmaster,blurry-amber-moth-dataset,grumpy-zucchini-kudu-kernel,chewy-myrtle-wasp-important,snappy-lemon-okapi-ordinal,cranky-eggplant-oyster-sumble,snappy-pear-quoll-unsorted,...,trippy-asparagus-bison-expert,lanky-aqua-bullfrog-important,stinky-olive-kiwi-golden,sickly-denim-oriole-important,gloppy-persimmon-bee-unsorted,stealthy-azure-gopher-hint,hasty-ivory-dragonfly-goose,flimsy-turquoise-fox-kernel,lousy-zucchini-seahorse-dummy,shabby-magenta-whippet-novice,chewy-lime-peccary-fimbus,nippy-burgundy-beagle-novice,gloppy-firebrick-moth-dataset,sleazy-russet-iguana-unsorted,surly-corn-tzu-kernel,skinny-myrtle-whippet-fepid,homey-tangerine-sloth-discard,blurry-green-dodo-important,skinny-champagne-mayfly-sorted,dorky-purple-kiwi-hint,beady-mauve-frog-distraction,surly-brass-maltese-ordinal,beady-asparagus-opossum-expert,beady-rust-impala-dummy,droopy-amethyst-dachshund-hint,homey-crimson-budgerigar-grandmaster,droopy-cardinal-impala-important,woozy-apricot-moose-hint,paltry-sapphire-labradoodle-dummy,crappy-carmine-eagle-entropy,greasy-magnolia-spider-grandmaster,crabby-carmine-flounder-sorted,skimpy-copper-fowl-grandmaster,hasty-seashell-woodpecker-hint,snappy-purple-bobcat-important,thirsty-carmine-corgi-ordinal,homely-auburn-reindeer-unsorted,crappy-beige-tiger-fepid,cranky-auburn-swan-novice,chewy-bistre-buzzard-expert,skinny-cyan-macaque-pembus,slimy-periwinkle-otter-expert,snazzy-burgundy-clam-novice,cozy-ochre-gorilla-gaussian,homey-sangria-wolfhound-dummy,snazzy-asparagus-hippopotamus-contributor,paltry-red-hamster-sorted,zippy-dandelion-insect-golden,baggy-coral-bandicoot-unsorted,goopy-lavender-wolverine-fimbus,wheezy-myrtle-mandrill-entropy,wiggy-lilac-lemming-sorted,gloppy-cerise-snail-contributor,woozy-silver-havanese-gaussian,jumpy-thistle-discus-sorted,muggy-turquoise-donkey-important,blurry-buff-hyena-entropy,bluesy-chocolate-kudu-fepid,gamy-white-monster-expert,target
0,707b395ecdcbb4dc2eabea00e4d1b179,-2.070654,1.018160,0.228643,0.857221,0.052271,0.230303,-6.385090,0.439369,-0.721946,-0.227027,0.575964,1.541908,1.745286,-0.624271,3.600958,1.176489,-0.182776,-0.228391,1.682263,-0.833236,-4.377688,-5.372410,-0.477742,-0.179005,-0.516475,0.127391,-0.857591,-0.461500,2.160303,-2.118371,0.515493,-1.201493,-0.027377,-1.154024,0.753204,-0.179651,-0.807341,-1.663626,0.893806,1.650000,0.309518,0.606264,-3.233735,1.731866,0.142344,-0.056539,-0.332911,0.001577,-0.155122,0.

- eda를 통해 찾은 'wheezy-copper-turtle-magic' 변수와 상호작용하는 모델생성

In [27]:
cols = [c for c in train.columns if c not in ['id', 'target','wheezy-copper-turtle-magic']]
oof = np.zeros(len(train))
preds = np.zeros(len(test))

for i in tqdm_notebook(range(512)):
    # 총512의 정수를 가진 'wheezy-copper-turtle-magic'변수를 통해 개별모델 생성
    train1 = train[train['wheezy-copper-turtle-magic']==i]
    test1 = test[test['wheezy-copper-turtle-magic']==i]
    idx1 = train1.index; idx2 = test1.index
    train1.reset_index(drop=True,inplace=True)

    
    skf = StratifiedKFold(n_splits=10, random_state=42)
    for train_index, test_index in skf.split(train1, train1['target']):
        # 로지스틱모델
        clf = LogisticRegression(solver='liblinear',penalty='l1',C=0.05)
        clf.fit(train1.loc[train_index][cols],train1.loc[train_index]['target'])
        oof[idx1[test_index]] = clf.predict_proba(train1.loc[test_index][cols])[:,1]
        preds[idx2] += clf.predict_proba(test1[cols])[:,1] / skf.n_splits
        
# oof 정확도
auc = roc_auc_score(train['target'],oof)
print('LR  scores CV =',round(auc,5))


LR  scores CV = 0.80378
